In [213]:
path = '../Energy_Final.csv'
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

# load data files
weather = pd.read_csv('../00_source_data/weather.csv').drop(columns = 'Unnamed: 0')
RPS = pd.read_csv('../00_source_data/StatesRPS.csv').drop(columns = ['Percentage', 'TargetYear'])
prices = pd.read_csv('../sourcePrices.csv').drop(columns = 'Unnamed: 0')
solar = pd.read_csv('../00_source_data/solar-pv-prices.csv')
co2 = pd.read_excel('../CO2.xlsx')
generation = pd.read_csv('../00_source_data/clean_energy_percent_processed.csv').drop(columns = 'Unnamed: 0')

In [214]:
# unify column names
RPS = RPS.rename(columns = {'State':'StateName', 'Abbreviation': 'State'})
prices = prices.rename(columns = {'year':'Year', 'coal':'CoalPrice', 'naturalGas':'NaturalGasPrice', 
                         'nuclear':'NuclearPrice', 'petroleum':'PetroleumPrice'})
weather = weather.rename(columns = {'State':'StateName', 'percipitation':'Percipitation'})
solar = solar.rename(columns = {'Solar PV Module Cost (2019 US$ per W)':'SolarPrice'})
generation = generation.rename(columns = {'YEAR':'Year', 'STATE':'State', 
                                          'total energy generation':'TotalEnergyGeneration', 
                                          'clean energy generation':'CleanEnergyGeneration',
                                          'clean_energy_percent':'CleanEnergyPercent'})

# modify CO2
def modify_co2(r):
    # return three columns dataframe
    state = [r['State']]
    years = list(r.drop(['State', 'Percent', 'Absolute']).index)
    data =  r.drop(['State', 'Percent', 'Absolute']).values
    df = pd.DataFrame({'StateName': state*len(years), 'CO2': data, 'Year':years})
    return df

co2 = co2.apply(lambda r: modify_co2(r), axis = 1)
co2 = reduce(lambda df1, df2: pd.concat([df1, df2]), co2)

print('RPS Columns: ' + str(RPS.columns))
print('Weather Columns: ' + str(weather.columns))
print('Prices Columns: ' + str(prices.columns))
print('CO2 Columns: ' + str(co2.columns))
print('Solar Columns: ' + str(solar.columns))
print('Generation Columns: ' + str(generation.columns))


RPS Columns: Index(['StateName', 'State', 'RPS', 'Enacted'], dtype='object')
Weather Columns: Index(['Year', 'Average_Temp', 'StateName', 'Percipitation'], dtype='object')
Prices Columns: Index(['Year', 'CoalPrice', 'NaturalGasPrice', 'NuclearPrice',
       'PetroleumPrice', 'State'],
      dtype='object')
CO2 Columns: Index(['StateName', 'CO2', 'Year'], dtype='object')
Solar Columns: Index(['Entity', 'Code', 'Year', 'SolarPrice'], dtype='object')
Generation Columns: Index(['Year', 'State', 'TotalEnergyGeneration', 'CleanEnergyGeneration',
       'CleanEnergyPercent'],
      dtype='object')


In [215]:
# merge Weather with RPS, this would add State variable to weather
# state removed: Iowa,Kansas,Montana,North Dakota,Oklahoma,South Dakota,Wisconsin,DC,Hawaii
df = weather.merge(RPS, on = 'StateName', how = 'inner')

# merge df with weather, this would add State variable to solar
df = df.merge(solar, on = 'Year', how = 'inner')

# merge df with co2
df = df.merge(co2, on = ['StateName', 'Year'], how = 'inner')

# merge df with prcies
df = df.merge(prices, on = ['State', 'Year'], how = 'inner')

# merge df with generation
df = df.merge(generation, on = ['State', 'Year'], how = 'inner')

In [216]:
df.to_csv(path)